<img src="https://fontmeme.com/permalink/200218/c07b0b73a35b3a92b323babc47a3f08a.png" alt="star-wars-font" border="0"></a>

# 

In [1]:
# #
# from IPython.display import YouTubeVideo
# YouTubeVideo(id='-bzWSJG93P8', width=1000, height=1000)

# <img src="https://fontmeme.com/permalink/200218/d512e714bfb440e12fe3c1a10e07e99a.png" alt="star-wars-font" border="0"></a>

## 1.1 Import Libraries

In [2]:
import json
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from pandas.io.json import json_normalize
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## 1.2 Define functions

Every function names the library that we will need to be able to use it (marked by an \*\), the reason to not run the library inside the function but to do it at the beginning is that we can not be sure that we will not need that library for anything else, so it is better to run it once, at the beginning at have it ready in case we will need another functionality of it; given that if we just run it inside the function it will not work if we reference it outside of it.

In [3]:
def request_get_result_api(url):
    '''
    *import json
    *import requests
    
    input: api url string
    output: json of api
    
    '''
    headers = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2)'
               + 'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
    response = requests.get(url, headers =headers)
    return flatten_data(response)

In [4]:
def flatten_data(response):
    results = response.json()
    normalize_results = json_normalize(results)
    return normalize_results

In [5]:
def title_columns(df):
    '''
    input: dataframe
    output: list of the column names with the first letter as caps
    '''
    col = [''.join(i).title() for i in df.columns]
    print(col)

In [6]:
def url_content(url):
    '''
    *import requests
    *from bs4 import BeautifulSoup
    
    -downloads
    -parses w/ BeautifulSoup
    -return url content

    input: URL
    output: contenido url
    '''
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")
    return soup

In [7]:
def json_iterated(url,last_page):
    lst = []
    for i in range(1,last_page):
        urls = url+str(i)
        flatten_data = request_get_result_api(urls)
        flatten_data_all = json_normalize(flatten_data.results[0])
        lst.append(flatten_data_all)
    data = pd.concat(lst, ignore_index=True)
    return data


In [8]:
def df_union(df, col, right_key, df_2, gby_1, gby_2):
    df = df.explode(col).merge(df_2, left_on= col, right_on = right_key)
    df = df.drop(col, axis = 1)
    df = df.rename(columns={'name':col})
    df2_col =[i for i in df.columns if i.islower()]
    df = df.drop(df2_col, axis = 1)
    df = df.groupby([gby_1, gby_2])[col].apply(', '.join).reset_index()
    return df

In [9]:
def make_clickable(x):
    '''
    Hace que el la columna seleccionada sea 'clickable' y abra una ventana nueva
    '''
    return '<a target="_blank" href="{}">{}</a>'.format(x, x)

In [10]:
def path_to_html(path):
    '''
    Convierte los links en tags html.
    '''
    return '<img src="'+ path + '" width="5000" >'

# <img src="https://fontmeme.com/permalink/200218/3b58153cffc57605bc48340079b07354.png" alt="star-wars-font" border="0"></a>

I have selected the: https://swapi.co/ which is the Stars Wars API. 
<br><br>
First thing we'll do is to run the selected API and see what it returns.

In [11]:
main_api = request_get_result_api('https://swapi.co/api/').transpose()
main_api

,0
people,https://swapi.co/api/people/
planets,https://swapi.co/api/planets/
films,https://swapi.co/api/films/
species,https://swapi.co/api/species/
vehicles,https://swapi.co/api/vehicles/
starships,https://swapi.co/api/starships/


We see the main API is conformed of 6 other different subAPIs: `people`, `planets`, `films`, `species`, `vehicles` and `starships`. 

For the this project I have selected the following subAPI 


`films`: https://swapi.co/api/films/

## 2.1 Pass API function

In [12]:
request_get_result_api('https://swapi.co/api/films/')

,count,next,previous,results
0,7,None,None,"[{'title': 'A New Hope', 'episode_id': 4, 'ope..."


We can see the function directly normalizes the results for us but we still have nested information so we will have to flatten the data once more.

<img src='./img/flatten_1_SW.jpg'/>

## 2.2 Normalize and Flatten the Data... again

In [13]:
star_wars_films = json_normalize(request_get_result_api('https://swapi.co/api/films/').results[0])

In [14]:
star_wars_films

,title,episode_id,opening_crawl,director,producer,release_date,characters,planets,starships,vehicles,species,created,edited,url
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/planets/2/, https://swap...","[https://swapi.co/api/starships/2/, https://sw...","[https://swapi.co/api/vehicles/4/, https://swa...","[https://swapi.co/api/species/5/, https://swap...",2014-12-10T14:23:31.880000Z,2015-04-11T09:46:52.774897Z,https://swapi.co/api/films/1/
1,Attack of the Clones,2,There is unrest in the Galactic\r\nSenate. Sev...,George Lucas,Rick McCallum,2002-05-16,"[https://swapi.co/api/people/2/, https://swapi...","[https://swapi.co/api/planets/8/, https://swap...","[https://swapi.co/api/starships/21/, https://s...","[https://swapi.co/api/vehicles/4/, https://swa...","[https://swapi.co/api/species/32/, https://swa...",2014-12-20T10:57:57.886000Z,2015-04-11T09:45:01.623982Z,https://swapi.co/api/films/5/
2,The Phantom Menace,1,Turmoil has engulfed the\r\nGalactic Republic....,George Lucas,Rick McCallum,1999-05-19,"[https://swapi.co/api/people/2/, https://swapi...","[https://swapi.co/api/planets/8/, https://swap...","[https://swapi.co/api/starships/40/, https://s...","[https://swapi.co/api/vehicles/33/, https://sw...","[https://swapi.co/api/species/1/, https://swap...",2014-12-19T16:52:55.740000Z,2015-04-11T09:45:18.689301Z,https://swapi.co/api/films/4/
3,Revenge of the Sith,3,War! The Republic is crumbling\r\nunder attack...,George Lucas,Rick McCallum,2005-05-19,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/planets/2/, https://swap...","[https://swapi.co/api/starships/48/, https://s...","[https://swapi.co/api/vehicles/33/, https://sw...","[https://swapi.co/api/species/19/, https://swa...",2014-12-20T18:49:38.403000Z,2015-04-11T09:45:44.862122Z,https://swapi.co/api/films/6/
4,Return of the Jedi,6,Luke Skywalker has returned to\r\nhis home pla...,Richard Marquand,"Howard G. Kazanjian, George Lucas, Rick McCallum",1983-05-25,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/planets/5/, https://swap...","[https://swapi.co/api/starships/15/, https://s...","[https://swapi.co/api/vehicles/8/, https://swa...","[https://swapi.co/api/species/1/, https://swap...",2014-12-18T10:39:33.255000Z,2015-04-11T09:46:05.220365Z,https://swapi.co/api/films/3/
5,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/planets/4/, https://swap...","[https://swapi.co/api/starships/15/, https://s...","[https://swapi.co/api/vehicles/8/, https://swa...","[https://swapi.co/api/species/6/, https://swap...",2014-12-12T11:26:24.656000Z,2017-04-19T10:57:29.544256Z,https://swapi.co/api/films/2/
6,The Force Awakens,7,Luke Skywalker has vanished.\r\nIn his absence...,J. J. Abrams,"Kathleen Kennedy, J. J. Abrams, Bryan Burk",2015-12-11,"[https://swapi.co/api/people/1/, https://swapi...",[https://swapi.co/api/planets/61/],"[https://swapi.co/api/starships/77/, https://s...",[],"[https://swapi.co/api/species/3/, https://swap...",2015-04-17T06:51:30.504780Z,2015-12-17T14:31:47.617768Z,https://swapi.co/api/films/7/


## 2.4 Header Cleaning and Organization

Once the data frame is not nested we will go ahead and check the headers for a better manipulation of the Data Set. We will pass the columns through the `title_columns` function that we have defined before. This function takes the headers and returns all the column names in lower letters starting with capital letter, as such: Xxxx

In [15]:
title_columns(star_wars_films)

['Title', 'Episode_Id', 'Opening_Crawl', 'Director', 'Producer', 'Release_Date', 'Characters', 'Planets', 'Starships', 'Vehicles', 'Species', 'Created', 'Edited', 'Url']


In this specific case we now which columns interest us: `Episode`, `Title`, `Characters`, `Planets`, `Starships`, `Vehicles`, `Species`. But for fun's sakes we will clean it up and organize it anyways.

In [16]:
star_wars_films.columns = ['Title', 'Episode', 'Opening Crawl', 'Director', 'Producer', 
                           'Release Date', 'Characters', 'Planets', 'Starships', 'Vehicles', 
                           'Species', 'Created', 'Edited', 'Url']
sw_column_order = ['Episode', 'Title', 'Characters', 'Planets', 'Starships', 'Vehicles', 'Species',
                           'Opening Crawl', 'Director', 'Producer', 'Release Date', 'Edited', 'Created', 'Url']

In [17]:
star_wars_films = star_wars_films[sw_column_order]

As we can see the headers have not only been "cleaned" but also grouped:

In [18]:
star_wars_films

,Episode,Title,Characters,Planets,Starships,Vehicles,Species,Opening Crawl,Director,Producer,Release Date,Edited,Created,Url
0,4,A New Hope,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/planets/2/, https://swap...","[https://swapi.co/api/starships/2/, https://sw...","[https://swapi.co/api/vehicles/4/, https://swa...","[https://swapi.co/api/species/5/, https://swap...",It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,2015-04-11T09:46:52.774897Z,2014-12-10T14:23:31.880000Z,https://swapi.co/api/films/1/
1,2,Attack of the Clones,"[https://swapi.co/api/people/2/, https://swapi...","[https://swapi.co/api/planets/8/, https://swap...","[https://swapi.co/api/starships/21/, https://s...","[https://swapi.co/api/vehicles/4/, https://swa...","[https://swapi.co/api/species/32/, https://swa...",There is unrest in the Galactic\r\nSenate. Sev...,George Lucas,Rick McCallum,2002-05-16,2015-04-11T09:45:01.623982Z,2014-12-20T10:57:57.886000Z,https://swapi.co/api/films/5/
2,1,The Phantom Menace,"[https://swapi.co/api/people/2/, https://swapi...","[https://swapi.co/api/planets/8/, https://swap...","[https://swapi.co/api/starships/40/, https://s...","[https://swapi.co/api/vehicles/33/, https://sw...","[https://swapi.co/api/species/1/, https://swap...",Turmoil has engulfed the\r\nGalactic Republic....,George Lucas,Rick McCallum,1999-05-19,2015-04-11T09:45:18.689301Z,2014-12-19T16:52:55.740000Z,https://swapi.co/api/films/4/
3,3,Revenge of the Sith,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/planets/2/, https://swap...","[https://swapi.co/api/starships/48/, https://s...","[https://swapi.co/api/vehicles/33/, https://sw...","[https://swapi.co/api/species/19/, https://swa...",War! The Republic is crumbling\r\nunder attack...,George Lucas,Rick McCallum,2005-05-19,2015-04-11T09:45:44.862122Z,2014-12-20T18:49:38.403000Z,https://swapi.co/api/films/6/
4,6,Return of the Jedi,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/planets/5/, https://swap...","[https://swapi.co/api/starships/15/, https://s...","[https://swapi.co/api/vehicles/8/, https://swa...","[https://swapi.co/api/species/1/, https://swap...",Luke Skywalker has returned to\r\nhis home pla...,Richard Marquand,"Howard G. Kazanjian, George Lucas, Rick McCallum",1983-05-25,2015-04-11T09:46:05.220365Z,2014-12-18T10:39:33.255000Z,https://swapi.co/api/films/3/
5,5,The Empire Strikes Back,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/planets/4/, https://swap...","[https://swapi.co/api/starships/15/, https://s...","[https://swapi.co/api/vehicles/8/, https://swa...","[https://swapi.co/api/species/6/, https://swap...",It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,2017-04-19T10:57:29.544256Z,2014-12-12T11:26:24.656000Z,https://swapi.co/api/films/2/
6,7,The Force Awakens,"[https://swapi.co/api/people/1/, https://swapi...",[https://swapi.co/api/planets/61/],"[https://swapi.co/api/starships/77/, https://s...",[],"[https://swapi.co/api/species/3/, https://swap...",Luke Skywalker has vanished.\r\nIn his absence...,J. J. Abrams,"Kathleen Kennedy, J. J. Abrams, Bryan Burk",2015-12-11,2015-12-17T14:31:47.617768Z,2015-04-17T06:51:30.504780Z,https://swapi.co/api/films/7/


As we mentioned before we already now which columns we want from this DF so we will drop the ones that are of no interest for us to have, keep in mind, that this are of no use what so ever for the objective of the project, I'm aware that it's good practice to drop the columns at the end; the columns are: `Opening Crawl`, `Director`, `Producer`, `Release Date`, `Edited`, `Created`, `Url`

In [19]:
un_cols = ['Opening Crawl', 'Director', 'Producer',
       'Release Date','Created', 'Edited', 'Url']
star_wars_films = star_wars_films.drop(un_cols , axis = 1) 

In [20]:
star_wars_films

,Episode,Title,Characters,Planets,Starships,Vehicles,Species
0,4,A New Hope,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/planets/2/, https://swap...","[https://swapi.co/api/starships/2/, https://sw...","[https://swapi.co/api/vehicles/4/, https://swa...","[https://swapi.co/api/species/5/, https://swap..."
1,2,Attack of the Clones,"[https://swapi.co/api/people/2/, https://swapi...","[https://swapi.co/api/planets/8/, https://swap...","[https://swapi.co/api/starships/21/, https://s...","[https://swapi.co/api/vehicles/4/, https://swa...","[https://swapi.co/api/species/32/, https://swa..."
2,1,The Phantom Menace,"[https://swapi.co/api/people/2/, https://swapi...","[https://swapi.co/api/planets/8/, https://swap...","[https://swapi.co/api/starships/40/, https://s...","[https://swapi.co/api/vehicles/33/, https://sw...","[https://swapi.co/api/species/1/, https://swap..."
3,3,Revenge of the Sith,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/planets/2/, https://swap...","[https://swapi.co/api/starships/48/, https://s...","[https://swapi.co/api/vehicles/33/, https://sw...","[https://swapi.co/api/species/19/, https://swa..."
4,6,Return of the Jedi,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/planets/5/, https://swap...","[https://swapi.co/api/starships/15/, https://s...","[https://swapi.co/api/vehicles/8/, https://swa...","[https://swapi.co/api/species/1/, https://swap..."
5,5,The Empire Strikes Back,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/planets/4/, https://swap...","[https://swapi.co/api/starships/15/, https://s...","[https://swapi.co/api/vehicles/8/, https://swa...","[https://swapi.co/api/species/6/, https://swap..."
6,7,The Force Awakens,"[https://swapi.co/api/people/1/, https://swapi...",[https://swapi.co/api/planets/61/],"[https://swapi.co/api/starships/77/, https://s...",[],"[https://swapi.co/api/species/3/, https://swap..."


As we can see the information of the columns `Characters`, `Planets`, `Starships`, `Vehicles` and `Species` is formed by lists, this will make it difficult for us to substitute the information of each series for the equivalent information from the corresponding DF. In the following section we will check the corresponding API/value of the columns.

# <img src="https://fontmeme.com/permalink/200218/efa1b7fc542f62267e7bad708cd5b123.png" alt="star-wars-font" border="0"></a>


If we check the main API we can see it is made up of links for different subAPIs. In section 2 we decided to work with the Film subAPI and we saw that for many of the columns the information referenced other subAPIs.

In [21]:
main_api

,0
people,https://swapi.co/api/people/
planets,https://swapi.co/api/planets/
films,https://swapi.co/api/films/
species,https://swapi.co/api/species/
vehicles,https://swapi.co/api/vehicles/
starships,https://swapi.co/api/starships/


## 3.1 SubAPIs

First we need to know how many how much information all of the subAPIs have so we will check all the JSONs to make sure that by the end we will have all the information. For that we will go through all the SubAPIs and for each of them we will:<br>
<br>
1) Check the JSONs <br>
2) Iterate through the JSON <br>
3) Create a DF for each SubAPI

The results will be the following DataSets:

### People to `sw_char`

`People`
<br>/api/people/?page=9
<br>"count": 87,
    <br>"next": null, 
    <br>"previous": "https://swapi.co/api/people/?page=8", 

In [22]:
request_get_result_api('https://swapi.co/api/people/?page=')['count'][0]

87

In [23]:
sw_char = json_iterated('https://swapi.co/api/people/?page=', 10)

In [24]:
sw_char

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.co/api/planets/1/,"[https://swapi.co/api/films/2/, https://swapi....",[https://swapi.co/api/species/1/],"[https://swapi.co/api/vehicles/14/, https://sw...","[https://swapi.co/api/starships/12/, https://s...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.co/api/people/1/
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,https://swapi.co/api/planets/1/,"[https://swapi.co/api/films/2/, https://swapi....",[https://swapi.co/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.co/api/people/2/
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,https://swapi.co/api/planets/8/,"[https://swapi.co/api/films/2/, https://swapi....",[https://swapi.co/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.co/api/people/3/
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,https://swapi.co/api/planets/1/,"[https://swapi.co/api/films/2/, https://swapi....",[https://swapi.co/api/species/1/],[],[https://swapi.co/api/starships/13/],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.co/api/people/4/
4,Leia Organa,150,49,brown,light,brown,19BBY,female,https://swapi.co/api/planets/2/,"[https://swapi.co/api/films/2/, https://swapi....",[https://swapi.co/api/species/1/],[https://swapi.co/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.co/api/people/5/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Rey,unknown,unknown,brown,light,hazel,unknown,female,https://swapi.co/api/planets/28/,[https://swapi.co/api/films/7/],[https://swapi.co/api/species/1/],[],[],2015-04-17T06:54:01.495077Z,2015-04-17T06:54:01.495128Z,https://swapi.co/api/people/85/
83,Poe Dameron,unknown,unknown,brown,light,brown,unknown,male,https://swapi.co/api/planets/28/,[https://swapi.co/api/films/7/],[https://swapi.co/api/species/1/],[],[https://swapi.co/api/starships/77/],2015-04-17T06:55:21.622786Z,2015-04-17T06:55:21.622835Z,https://swapi.co/api/people/86/
84,BB8,unknown,unknown,none,none,black,unknown,none,https://swapi.co/api/planets/28/,[https://swapi.co/api/films/7/],[https://swapi.co/api/species/2/],[],[],2015-04-17T06:57:38.061346Z,2015-04-17T06:57:38.061453Z,https://swapi.co/api/people/87/
85,Captain Phasma,unknown,unknown,unknown,unknown,unknown,unknown,female,https://swapi.co/api/planets/28/,[https://swapi.co/api/films/7/],[],[],[],2015-10-13T10:35:39.229823Z,2015-10-13T10:35:39.229894Z,https://swapi.co/api/people/88/


### Planets to `sw_planets`

`Planets`
<br>/api/planets/?page=7
<br>"count": 61, 
<br>"next": null, 
<br>"previous": "https://swapi.co/api/planets/?page=6", 

In [25]:
request_get_result_api('https://swapi.co/api/planets/?page=')['count'][0]

61

In [26]:
sw_planets = json_iterated('https://swapi.co/api/planets/?page=', 8)

In [27]:
sw_planets

,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url
0,Alderaan,24,364,12500,temperate,1 standard,"grasslands, mountains",40,2000000000,"[https://swapi.co/api/people/5/, https://swapi...","[https://swapi.co/api/films/6/, https://swapi....",2014-12-10T11:35:48.479000Z,2014-12-20T20:58:18.420000Z,https://swapi.co/api/planets/2/
1,Yavin IV,24,4818,10200,"temperate, tropical",1 standard,"jungle, rainforests",8,1000,[],[https://swapi.co/api/films/1/],2014-12-10T11:37:19.144000Z,2014-12-20T20:58:18.421000Z,https://swapi.co/api/planets/3/
2,Hoth,23,549,7200,frozen,1.1 standard,"tundra, ice caves, mountain ranges",100,unknown,[],[https://swapi.co/api/films/2/],2014-12-10T11:39:13.934000Z,2014-12-20T20:58:18.423000Z,https://swapi.co/api/planets/4/
3,Dagobah,23,341,8900,murky,N/A,"swamp, jungles",8,unknown,[],"[https://swapi.co/api/films/2/, https://swapi....",2014-12-10T11:42:22.590000Z,2014-12-20T20:58:18.425000Z,https://swapi.co/api/planets/5/
4,Bespin,12,5110,118000,temperate,"1.5 (surface), 1 standard (Cloud City)",gas giant,0,6000000,[https://swapi.co/api/people/26/],[https://swapi.co/api/films/2/],2014-12-10T11:43:55.240000Z,2014-12-20T20:58:18.427000Z,https://swapi.co/api/planets/6/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Shili,unknown,unknown,unknown,temperate,1,"cities, savannahs, seas, plains",unknown,unknown,[https://swapi.co/api/people/78/],[],2014-12-20T18:43:14.049000Z,2014-12-20T20:58:18.521000Z,https://swapi.co/api/planets/58/
57,Kalee,23,378,13850,"arid, temperate, tropical",1,"rainforests, cliffs, canyons, seas",unknown,4000000000,[https://swapi.co/api/people/79/],[],2014-12-20T19:43:51.278000Z,2014-12-20T20:58:18.523000Z,https://swapi.co/api/planets/59/
58,Umbara,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,[https://swapi.co/api/people/82/],[],2014-12-20T20:18:36.256000Z,2014-12-20T20:58:18.525000Z,https://swapi.co/api/planets/60/
59,Tatooine,23,304,10465,arid,1 standard,desert,1,200000,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/films/5/, https://swapi....",2014-12-09T13:50:49.641000Z,2014-12-21T20:48:04.175778Z,https://swapi.co/api/planets/1/


### Species to `sw_species`

`Species`
<br>/api/species/?page=4
<br>"count": 37, 
<br>"next": null, 
<br>"previous": "https://swapi.co/api/species/?page=3", 

In [28]:
request_get_result_api('https://swapi.co/api/species/?page=')['count'][0]

37

In [29]:
sw_species = json_iterated('https://swapi.co/api/species/?page=', 5)

In [30]:
sw_species

,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld,language,people,films,created,edited,url
0,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000,https://swapi.co/api/planets/24/,Huttese,[https://swapi.co/api/people/16/],"[https://swapi.co/api/films/3/, https://swapi....",2014-12-10T17:12:50.410000Z,2014-12-20T21:36:42.146000Z,https://swapi.co/api/species/5/
1,Yoda's species,mammal,sentient,66,"green, yellow","brown, white","brown, green, yellow",900,https://swapi.co/api/planets/28/,Galactic basic,[https://swapi.co/api/people/20/],"[https://swapi.co/api/films/2/, https://swapi....",2014-12-15T12:27:22.877000Z,2014-12-20T21:36:42.148000Z,https://swapi.co/api/species/6/
2,Trandoshan,reptile,sentient,200,"brown, green",none,"yellow, orange",unknown,https://swapi.co/api/planets/29/,Dosh,[https://swapi.co/api/people/24/],[https://swapi.co/api/films/2/],2014-12-15T13:07:47.704000Z,2014-12-20T21:36:42.151000Z,https://swapi.co/api/species/7/
3,Mon Calamari,amphibian,sentient,160,"red, blue, brown, magenta",none,yellow,unknown,https://swapi.co/api/planets/31/,Mon Calamarian,[https://swapi.co/api/people/27/],[https://swapi.co/api/films/3/],2014-12-18T11:09:52.263000Z,2014-12-20T21:36:42.153000Z,https://swapi.co/api/species/8/
4,Ewok,mammal,sentient,100,brown,"white, brown, black","orange, brown",unknown,https://swapi.co/api/planets/7/,Ewokese,[https://swapi.co/api/people/30/],[https://swapi.co/api/films/3/],2014-12-18T11:22:00.285000Z,2014-12-20T21:36:42.155000Z,https://swapi.co/api/species/9/
5,Sullustan,mammal,sentient,180,pale,none,black,unknown,https://swapi.co/api/planets/33/,Sullutese,[https://swapi.co/api/people/31/],[https://swapi.co/api/films/3/],2014-12-18T11:26:20.103000Z,2014-12-20T21:36:42.157000Z,https://swapi.co/api/species/10/
6,Neimodian,unknown,sentient,180,"grey, green",none,"red, pink",unknown,https://swapi.co/api/planets/18/,Neimoidia,[https://swapi.co/api/people/33/],[https://swapi.co/api/films/4/],2014-12-19T17:07:31.319000Z,2014-12-20T21:36:42.160000Z,https://swapi.co/api/species/11/
7,Gungan,amphibian,sentient,190,"brown, green",none,orange,unknown,https://swapi.co/api/planets/8/,Gungan basic,"[https://swapi.co/api/people/36/, https://swap...","[https://swapi.co/api/films/5/, https://swapi....",2014-12-19T17:30:37.341000Z,2014-12-20T21:36:42.163000Z,https://swapi.co/api/species/12/
8,Toydarian,mammal,sentient,120,"blue, green, grey",none,yellow,91,https://swapi.co/api/planets/34/,Toydarian,[https://swapi.co/api/people/40/],"[https://swapi.co/api/films/5/, https://swapi....",2014-12-19T17:48:56.893000Z,2014-12-20T21:36:42.165000Z,https://swapi.co/api/species/13/
9,Dug,mammal,sentient,100,"brown, purple, grey, red",none,"yellow, blue",unknown,https://swapi.co/api/planets/35/,Dugese,[https://swapi.co/api/people/41/],[https://swapi.co/api/films/4/],2014-12-19T17:53:11.214000Z,2014-12-20T21:36:42.167000Z,https://swapi.co/api/species/14/


### Vehicles to `sw_vehicles`

`Vehicles`
<br>/api/vehicles/?page=4
<br>"count": 39, 
<br>"next": null, 
<br>"previous": "https://swapi.co/api/vehicles/?page=3", 

In [31]:
request_get_result_api('https://swapi.co/api/vehicles/?page=')['count'][0]

39

In [32]:
sw_vehicles = json_iterated('https://swapi.co/api/vehicles/?page=', 5)

In [33]:
sw_vehicles

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,pilots,films,created,edited,url
0,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,[],"[https://swapi.co/api/films/5/, https://swapi....",2014-12-10T15:36:25.724000Z,2014-12-22T18:21:15.523587Z,https://swapi.co/api/vehicles/4/
1,T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft,[],[https://swapi.co/api/films/1/],2014-12-10T16:01:52.434000Z,2014-12-22T18:21:15.552614Z,https://swapi.co/api/vehicles/6/
2,X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,unknown,repulsorcraft,[],[https://swapi.co/api/films/1/],2014-12-10T16:13:52.586000Z,2014-12-22T18:21:15.583700Z,https://swapi.co/api/vehicles/7/
3,TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,unknown,6.4,1200,1,0,65,2 days,starfighter,[],"[https://swapi.co/api/films/2/, https://swapi....",2014-12-10T16:33:52.860000Z,2014-12-22T18:21:15.606149Z,https://swapi.co/api/vehicles/8/
4,Snowspeeder,t-47 airspeeder,Incom corporation,unknown,4.5,650,2,0,10,none,airspeeder,"[https://swapi.co/api/people/1/, https://swapi...",[https://swapi.co/api/films/2/],2014-12-15T12:22:12Z,2014-12-22T18:21:15.623033Z,https://swapi.co/api/vehicles/14/
5,TIE bomber,TIE/sa bomber,Sienar Fleet Systems,unknown,7.8,850,1,0,none,2 days,space/planetary bomber,[],"[https://swapi.co/api/films/2/, https://swapi....",2014-12-15T12:33:15.838000Z,2014-12-22T18:21:15.667730Z,https://swapi.co/api/vehicles/16/
6,AT-AT,All Terrain Armored Transport,"Kuat Drive Yards, Imperial Department of Milit...",unknown,20,60,5,40,1000,unknown,assault walker,[],"[https://swapi.co/api/films/2/, https://swapi....",2014-12-15T12:38:25.937000Z,2014-12-22T18:21:15.714673Z,https://swapi.co/api/vehicles/18/
7,AT-ST,All Terrain Scout Transport,"Kuat Drive Yards, Imperial Department of Milit...",unknown,2,90,2,0,200,none,walker,[https://swapi.co/api/people/13/],"[https://swapi.co/api/films/2/, https://swapi....",2014-12-15T12:46:42.384000Z,2014-12-22T18:21:15.761584Z,https://swapi.co/api/vehicles/19/
8,Storm IV Twin-Pod cloud car,Storm IV Twin-Pod,Bespin Motors,75000,7,1500,2,0,10,1 day,repulsorcraft,[],[https://swapi.co/api/films/2/],2014-12-15T12:58:50.530000Z,2014-12-22T18:21:15.783232Z,https://swapi.co/api/vehicles/20/
9,Sail barge,Modified Luxury Sail Barge,Ubrikkian Industries Custom Vehicle Division,285000,30,100,26,500,2000000,Live food tanks,sail barge,[],[https://swapi.co/api/films/3/],2014-12-18T10:44:14.217000Z,2014-12-22T18:21:15.807906Z,https://swapi.co/api/vehicles/24/


### Starships to `sw_starships`

`Starships`
<br>/api/starships/?page=4
<br>"count": 37, 
<br>"next": null, 
<br>"previous": "https://swapi.co/api/starships/?page=3", 

In [34]:
request_get_result_api('https://swapi.co/api/starships/?page=')['count'][0]

37

In [35]:
sw_starships = json_iterated('https://swapi.co/api/starships/?page=', 5)

In [36]:
sw_starships

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films,created,edited,url
0,Executor,Executor-class star dreadnought,"Kuat Drive Yards, Fondor Shipyards",1143350000,19000,n/a,279144,38000,250000000,6 years,2.0,40,Star dreadnought,[],"[https://swapi.co/api/films/2/, https://swapi....",2014-12-15T12:31:42.547000Z,2017-04-19T10:56:06.685592Z,https://swapi.co/api/starships/15/
1,Sentinel-class landing craft,Sentinel-class landing craft,"Sienar Fleet Systems, Cyngus Spaceworks",240000,38,1000,5,75,180000,1 month,1.0,70,landing craft,[],[https://swapi.co/api/films/1/],2014-12-10T15:48:00.586000Z,2014-12-22T17:35:44.431407Z,https://swapi.co/api/starships/5/
2,Death Star,DS-1 Orbital Battle Station,"Imperial Department of Military Research, Sien...",1000000000000,120000,n/a,342953,843342,1000000000000,3 years,4.0,10,Deep Space Mobile Battlestation,[],[https://swapi.co/api/films/1/],2014-12-10T16:36:50.509000Z,2014-12-22T17:35:44.452589Z,https://swapi.co/api/starships/9/
3,Millennium Falcon,YT-1300 light freighter,Corellian Engineering Corporation,100000,34.37,1050,4,6,100000,2 months,0.5,75,Light freighter,"[https://swapi.co/api/people/13/, https://swap...","[https://swapi.co/api/films/2/, https://swapi....",2014-12-10T16:59:45.094000Z,2014-12-22T17:35:44.464156Z,https://swapi.co/api/starships/10/
4,Y-wing,BTL Y-wing,Koensayr Manufacturing,134999,14,1000km,2,0,110,1 week,1.0,80,assault starfighter,[],"[https://swapi.co/api/films/2/, https://swapi....",2014-12-12T11:00:39.817000Z,2014-12-22T17:35:44.479706Z,https://swapi.co/api/starships/11/
5,X-wing,T-65 X-wing,Incom Corporation,149999,12.5,1050,1,0,110,1 week,1.0,100,Starfighter,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/films/2/, https://swapi....",2014-12-12T11:19:05.340000Z,2014-12-22T17:35:44.491233Z,https://swapi.co/api/starships/12/
6,TIE Advanced x1,Twin Ion Engine Advanced x1,Sienar Fleet Systems,unknown,9.2,1200,1,0,150,5 days,1.0,105,Starfighter,[https://swapi.co/api/people/4/],[https://swapi.co/api/films/1/],2014-12-12T11:21:32.991000Z,2014-12-22T17:35:44.549047Z,https://swapi.co/api/starships/13/
7,Slave 1,Firespray-31-class patrol and attack,Kuat Systems Engineering,unknown,21.5,1000,1,6,70000,1 month,3.0,70,Patrol craft,[https://swapi.co/api/people/22/],"[https://swapi.co/api/films/2/, https://swapi....",2014-12-15T13:00:56.332000Z,2014-12-22T17:35:44.716273Z,https://swapi.co/api/starships/21/
8,Imperial shuttle,Lambda-class T-4a shuttle,Sienar Fleet Systems,240000,20,850,6,20,80000,2 months,1.0,50,Armed government transport,"[https://swapi.co/api/people/1/, https://swapi...","[https://swapi.co/api/films/2/, https://swapi....",2014-12-15T13:04:47.235000Z,2014-12-22T17:35:44.795405Z,https://swapi.co/api/starships/22/
9,EF76 Nebulon-B escort frigate,EF76 Nebulon-B escort frigate,Kuat Drive Yards,8500000,300,800,854,75,6000000,2 years,2.0,40,Escort ship,[],"[https://swapi.co/api/films/2/, https://swapi....",2014-12-15T13:06:30.813000Z,2014-12-22T17:35:44.848329Z,https://swapi.co/api/starships/23/


## 4. Combine DF and GroupBy Rows

### 4.1 `star_wars_films_char` +  `star_wars_films_planets`  = `star_wars_films_charnets`

In [37]:
star_wars_films_char = df_union(star_wars_films,'Characters', 'url', sw_char, 'Title', 'Episode')
star_wars_films_char

,Title,Episode,Characters
0,A New Hope,4,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei..."
1,Attack of the Clones,2,"C-3PO, R2-D2, Owen Lars, Beru Whitesun lars, O..."
2,Return of the Jedi,6,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei..."
3,Revenge of the Sith,3,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei..."
4,The Empire Strikes Back,5,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei..."
5,The Force Awakens,7,"Luke Skywalker, R2-D2, Leia Organa, Chewbacca,..."
6,The Phantom Menace,1,"C-3PO, R2-D2, Obi-Wan Kenobi, Jabba Desilijic ..."


In [38]:
star_wars_films_planets = df_union(star_wars_films,'Planets','url',sw_planets, 'Title', 'Episode')
star_wars_films_planets

,Title,Episode,Planets
0,A New Hope,4,"Alderaan, Yavin IV, Tatooine"
1,Attack of the Clones,2,"Tatooine, Naboo, Coruscant, Kamino, Geonosis"
2,Return of the Jedi,6,"Tatooine, Naboo, Coruscant, Dagobah, Endor"
3,Revenge of the Sith,3,"Alderaan, Tatooine, Naboo, Coruscant, Dagobah,..."
4,The Empire Strikes Back,5,"Dagobah, Hoth, Bespin, Ord Mantell"
5,The Force Awakens,7,Jakku
6,The Phantom Menace,1,"Tatooine, Naboo, Coruscant"


In [39]:
star_wars_films_charnets = star_wars_films_char.merge(star_wars_films_planets, on= ['Episode','Title'])
star_wars_films_charnets

,Title,Episode,Characters,Planets
0,A New Hope,4,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Alderaan, Yavin IV, Tatooine"
1,Attack of the Clones,2,"C-3PO, R2-D2, Owen Lars, Beru Whitesun lars, O...","Tatooine, Naboo, Coruscant, Kamino, Geonosis"
2,Return of the Jedi,6,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Tatooine, Naboo, Coruscant, Dagobah, Endor"
3,Revenge of the Sith,3,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Alderaan, Tatooine, Naboo, Coruscant, Dagobah,..."
4,The Empire Strikes Back,5,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Dagobah, Hoth, Bespin, Ord Mantell"
5,The Force Awakens,7,"Luke Skywalker, R2-D2, Leia Organa, Chewbacca,...",Jakku
6,The Phantom Menace,1,"C-3PO, R2-D2, Obi-Wan Kenobi, Jabba Desilijic ...","Tatooine, Naboo, Coruscant"


### 4.2 `star_wars_films_vehicles`  +  `star_wars_films_starships`  = `star_wars_films_vroom`

In [40]:
star_wars_films_vehicles = df_union(star_wars_films, 'Vehicles','url',sw_vehicles,'Title', 'Episode')
star_wars_films_vehicles

,Title,Episode,Vehicles
0,A New Hope,4,"Sand Crawler, T-16 skyhopper, X-34 landspeeder..."
1,Attack of the Clones,2,"Sand Crawler, Zephyr-G swoop bike, Koro-2 Exod..."
2,Return of the Jedi,6,"TIE/LN starfighter, TIE bomber, AT-AT, AT-ST, ..."
3,Revenge of the Sith,3,"LAAT/i, AT-TE, Neimoidian shuttle, Vulture Dro..."
4,The Empire Strikes Back,5,"TIE/LN starfighter, TIE bomber, AT-AT, AT-ST, ..."
5,The Phantom Menace,1,"Vulture Droid, Multi-Troop Transport, Armored ..."


In [41]:
star_wars_films_starships = df_union(star_wars_films,'Starships', 'url', sw_starships, 'Title', 'Episode')
star_wars_films_starships

,Title,Episode,Starships
0,A New Hope,4,"CR90 corvette, Star Destroyer, Sentinel-class ..."
1,Attack of the Clones,2,"Slave 1, Naboo fighter, J-type diplomatic barg..."
2,Return of the Jedi,6,"CR90 corvette, Star Destroyer, Millennium Falc..."
3,Revenge of the Sith,3,"CR90 corvette, Jedi starfighter, Droid control..."
4,The Empire Strikes Back,5,"Star Destroyer, Millennium Falcon, Y-wing, X-w..."
5,The Force Awakens,7,"Millennium Falcon, T-70 X-wing fighter"
6,The Phantom Menace,1,"Naboo fighter, Droid control ship, Naboo Royal..."


In [42]:
#on the merge parameters the second one is 'how' and we could do: 'left', because the vehicles df is only 6 rows 
#if we don't do a left merge we will loose the row where Vehicles = NaN, given that we will focus on the original
#Star Wars movies we will ignore anythin that comes after episode 6, but it would be:
#star_wars_films_starships.\merge(star_wars_films_vehicles, 'left', on= ['Episode','Title'])
star_wars_films_vroom = star_wars_films_starships.\
                        merge(star_wars_films_vehicles, on= ['Episode','Title'])
star_wars_films_vroom

,Title,Episode,Starships,Vehicles
0,A New Hope,4,"CR90 corvette, Star Destroyer, Sentinel-class ...","Sand Crawler, T-16 skyhopper, X-34 landspeeder..."
1,Attack of the Clones,2,"Slave 1, Naboo fighter, J-type diplomatic barg...","Sand Crawler, Zephyr-G swoop bike, Koro-2 Exod..."
2,Return of the Jedi,6,"CR90 corvette, Star Destroyer, Millennium Falc...","TIE/LN starfighter, TIE bomber, AT-AT, AT-ST, ..."
3,Revenge of the Sith,3,"CR90 corvette, Jedi starfighter, Droid control...","LAAT/i, AT-TE, Neimoidian shuttle, Vulture Dro..."
4,The Empire Strikes Back,5,"Star Destroyer, Millennium Falcon, Y-wing, X-w...","TIE/LN starfighter, TIE bomber, AT-AT, AT-ST, ..."
5,The Phantom Menace,1,"Naboo fighter, Droid control ship, Naboo Royal...","Vulture Droid, Multi-Troop Transport, Armored ..."


### 4.3 `star_wars_films_charnets` +  `star_wars_films_vroom` = `star_wars_films_final`

In [43]:
star_wars_films_final = star_wars_films_charnets.merge(star_wars_films_vroom, on= ['Episode','Title'])
star_wars_films_final

,Title,Episode,Characters,Planets,Starships,Vehicles
0,A New Hope,4,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Alderaan, Yavin IV, Tatooine","CR90 corvette, Star Destroyer, Sentinel-class ...","Sand Crawler, T-16 skyhopper, X-34 landspeeder..."
1,Attack of the Clones,2,"C-3PO, R2-D2, Owen Lars, Beru Whitesun lars, O...","Tatooine, Naboo, Coruscant, Kamino, Geonosis","Slave 1, Naboo fighter, J-type diplomatic barg...","Sand Crawler, Zephyr-G swoop bike, Koro-2 Exod..."
2,Return of the Jedi,6,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Tatooine, Naboo, Coruscant, Dagobah, Endor","CR90 corvette, Star Destroyer, Millennium Falc...","TIE/LN starfighter, TIE bomber, AT-AT, AT-ST, ..."
3,Revenge of the Sith,3,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Alderaan, Tatooine, Naboo, Coruscant, Dagobah,...","CR90 corvette, Jedi starfighter, Droid control...","LAAT/i, AT-TE, Neimoidian shuttle, Vulture Dro..."
4,The Empire Strikes Back,5,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Dagobah, Hoth, Bespin, Ord Mantell","Star Destroyer, Millennium Falcon, Y-wing, X-w...","TIE/LN starfighter, TIE bomber, AT-AT, AT-ST, ..."
5,The Phantom Menace,1,"C-3PO, R2-D2, Obi-Wan Kenobi, Jabba Desilijic ...","Tatooine, Naboo, Coruscant","Naboo fighter, Droid control ship, Naboo Royal...","Vulture Droid, Multi-Troop Transport, Armored ..."


### 4.4 `star_wars_films_species` &  `star_wars_films` 

In [44]:
#import matplotlib

In [45]:
star_wars_films_species = df_union(star_wars_films, 'Species', 'url', sw_species, 'Title', 'Episode')
star_wars_films_species

,Title,Episode,Species
0,A New Hope,4,"Hutt, Wookiee, Droid, Human, Rodian"
1,Attack of the Clones,2,"Droid, Human, Kaminoan, Skakoan, Togruta, Yoda..."
2,Return of the Jedi,6,"Hutt, Wookiee, Droid, Human, Yoda's species, T..."
3,Revenge of the Sith,3,"Wookiee, Droid, Human, Skakoan, Togruta, Yoda'..."
4,The Empire Strikes Back,5,"Wookiee, Droid, Human, Yoda's species, Trandoshan"
5,The Force Awakens,7,"Wookiee, Droid, Human"
6,The Phantom Menace,1,"Droid, Human, Yoda's species, Gungan, Toydaria..."


In [46]:
star_wars_films = star_wars_films_final.merge(star_wars_films_species, on= ['Episode','Title'])
star_wars_films

,Title,Episode,Characters,Planets,Starships,Vehicles,Species
0,A New Hope,4,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Alderaan, Yavin IV, Tatooine","CR90 corvette, Star Destroyer, Sentinel-class ...","Sand Crawler, T-16 skyhopper, X-34 landspeeder...","Hutt, Wookiee, Droid, Human, Rodian"
1,Attack of the Clones,2,"C-3PO, R2-D2, Owen Lars, Beru Whitesun lars, O...","Tatooine, Naboo, Coruscant, Kamino, Geonosis","Slave 1, Naboo fighter, J-type diplomatic barg...","Sand Crawler, Zephyr-G swoop bike, Koro-2 Exod...","Droid, Human, Kaminoan, Skakoan, Togruta, Yoda..."
2,Return of the Jedi,6,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Tatooine, Naboo, Coruscant, Dagobah, Endor","CR90 corvette, Star Destroyer, Millennium Falc...","TIE/LN starfighter, TIE bomber, AT-AT, AT-ST, ...","Hutt, Wookiee, Droid, Human, Yoda's species, T..."
3,Revenge of the Sith,3,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Alderaan, Tatooine, Naboo, Coruscant, Dagobah,...","CR90 corvette, Jedi starfighter, Droid control...","LAAT/i, AT-TE, Neimoidian shuttle, Vulture Dro...","Wookiee, Droid, Human, Skakoan, Togruta, Yoda'..."
4,The Empire Strikes Back,5,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Dagobah, Hoth, Bespin, Ord Mantell","Star Destroyer, Millennium Falcon, Y-wing, X-w...","TIE/LN starfighter, TIE bomber, AT-AT, AT-ST, ...","Wookiee, Droid, Human, Yoda's species, Trandoshan"
5,The Phantom Menace,1,"C-3PO, R2-D2, Obi-Wan Kenobi, Jabba Desilijic ...","Tatooine, Naboo, Coruscant","Naboo fighter, Droid control ship, Naboo Royal...","Vulture Droid, Multi-Troop Transport, Armored ...","Droid, Human, Yoda's species, Gungan, Toydaria..."


## 5. Finalizing the Data Set from the API 

Once all we have extracted all the information we organize by episode, for future reasons we will set it `Episode` as an index and then we will reset it back ti the regular index. 

In [47]:
sw_final = star_wars_films_final.set_index('Episode')
star_wars_final = sw_final.sort_values(by = 'Episode', ascending = True)
star_wars_final = star_wars_final.reset_index()
star_wars_final

,Episode,Title,Characters,Planets,Starships,Vehicles
0,1,The Phantom Menace,"C-3PO, R2-D2, Obi-Wan Kenobi, Jabba Desilijic ...","Tatooine, Naboo, Coruscant","Naboo fighter, Droid control ship, Naboo Royal...","Vulture Droid, Multi-Troop Transport, Armored ..."
1,2,Attack of the Clones,"C-3PO, R2-D2, Owen Lars, Beru Whitesun lars, O...","Tatooine, Naboo, Coruscant, Kamino, Geonosis","Slave 1, Naboo fighter, J-type diplomatic barg...","Sand Crawler, Zephyr-G swoop bike, Koro-2 Exod..."
2,3,Revenge of the Sith,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Alderaan, Tatooine, Naboo, Coruscant, Dagobah,...","CR90 corvette, Jedi starfighter, Droid control...","LAAT/i, AT-TE, Neimoidian shuttle, Vulture Dro..."
3,4,A New Hope,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Alderaan, Yavin IV, Tatooine","CR90 corvette, Star Destroyer, Sentinel-class ...","Sand Crawler, T-16 skyhopper, X-34 landspeeder..."
4,5,The Empire Strikes Back,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Dagobah, Hoth, Bespin, Ord Mantell","Star Destroyer, Millennium Falcon, Y-wing, X-w...","TIE/LN starfighter, TIE bomber, AT-AT, AT-ST, ..."
5,6,Return of the Jedi,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Lei...","Tatooine, Naboo, Coruscant, Dagobah, Endor","CR90 corvette, Star Destroyer, Millennium Falc...","TIE/LN starfighter, TIE bomber, AT-AT, AT-ST, ..."


# <img src="https://fontmeme.com/permalink/200218/4ff347772acdab6d2d95fc087b8f4211.png" alt="star-wars-font" border="0"></a>

From the official **Star Wars** website we will take the movie _thumbnails_ and the _links_ for each of the movies and add it to the previously created columns: `Thumbnails` and `Official Site`.

## Getting the content

In [48]:
url_content('https://www.starwars.com/films')
official_sw = url_content('https://www.starwars.com/films')

<!DOCTYPE html>
<!--

    "May the Source be with you"

--><!--[if lt IE 9]><html class="no-js no-media-query"><![endif]--><!--[if (gte IE 9)|(!IE)]><!--><html class="no-js" lang="en" version="HTML+RDFa 1.1">
<!--<![endif]-->
<head prefix="fb: http://ogp.me/ns/fb# og: http://ogp.me/ns#">
<title>Star Wars Movies | StarWars.com</title>
<link href="https://static-mh.content.disney.io/starwars/assets/application-4f9eb08db948.css" rel="stylesheet" type="text/css"/>
<link href="https://static-mh.content.disney.io/starwars/assets/modals-a044a0ab12e0.css" rel="stylesheet" type="text/css"/>
<link href="https://static-mh.content.disney.io/starwars/assets/modules-c41cd12327d0.css" rel="stylesheet" type="text/css"/>
<link href="https://static-mh.content.disney.io/starwars/assets/portals/www.starwars.com/application-d41d8cd98f00.css" rel="stylesheet" type="text/css"/>
<link href="https://static-mh.content.disney.io/matterhorn/assets/goc/basic-036659325618.css" rel="stylesheet" type="text/css"/><scr

### `Thumbnail` scraping

In [49]:
imags =[]
for link in official_sw.find_all('img', {'class':"thumb reserved-ratio"}):
    imags.append(link['src'])
    
promo_poster = []
for x in imags:
    c = "".join(x)
    if 'Star-Wars' in c:
        promo_poster.append(c)

promo_poster_ordered = promo_poster[::-1]
promo_poster_ordered 

['https://lumiere-a.akamaihd.net/v1/images/Star-Wars-Phantom-Menace-I-Poster_f5832812.jpeg?region=0%2C250%2C678%2C340&width=320',
 'https://lumiere-a.akamaihd.net/v1/images/Star-Wars-Attack-Clones-II-Poster_53baa2e7.jpeg?region=0%2C188%2C678%2C340&width=320',
 'https://lumiere-a.akamaihd.net/v1/images/Star-Wars-Revenge-Sith-III-Poster_646108ce.jpeg?region=0%2C356%2C746%2C374&width=320',
 'https://lumiere-a.akamaihd.net/v1/images/Star-Wars-New-Hope-IV-Poster_c217085b.jpeg?region=40%2C219%2C586%2C293&width=320',
 'https://lumiere-a.akamaihd.net/v1/images/Star-Wars-Empire-Strikes-Back-V-Poster_878f7fce.jpeg?region=25%2C299%2C612%2C306&width=320',
 'https://lumiere-a.akamaihd.net/v1/images/Star-Wars-Return-Jedi-VI-Poster_a10501d2.jpeg?region=9%2C210%2C624%2C312&width=320']

### `Official Site` scraping

This are not the `href` you are looking for, we will find a lot of links from where we will have to "take" the ones we need. 

In [50]:
links = [link.get('href') for link in official_sw.find_all('a', href=True)]
links

['https://www.disney.com/',
 'https://www.disneyplus.com/?cid=DTCI-Synergy-Disneycom-Site-NA-DisneyPlus-US-DisneyPlus-DisneyPlus-EN-GlobalNav-Desktop',
 'https://disneyplusoriginals.disney.com/',
 'https://www.shopdisney.com?cmp=OTL-Dcom_ChromShpIconB_Shop_EFC28055',
 'https://www.shopdisney.com?cmp=OTL-Dcom_att_DropRollover_Sale_EFC280559',
 'https://www.shopdisney.com/clothing-content?CMP=OTL-Dcom_att_DropRollover_Clothes_EFC280559',
 'https://www.shopdisney.com/accessories-content?CMP=OTL-Dcom_att_DropRollover_Accessories_EFC280559',
 'https://www.shopdisney.com/toys-content?CMP=OTL-Dcom_att_DropRollover_Toys_EFC280559',
 'https://www.shopdisney.com/?CMP=OTL-Dcom_att_DropRollover_AllStore_EFC_280559',
 'https://disneyparks.disney.go.com/?DISCID=DI_mtt_chrome',
 'https://disneyworld.disney.go.com/?DISCID=DI_mtt_chrome_dropdown_wdw',
 'https://disneyland.disney.go.com/?DISCID=DI_mtt_chrome_dropdown_dlr',
 'https://disneycruise.disney.go.com/?DISCID=DI_mtt_chrome_dropdown_dcl',
 'https

To selecte the 'href' we are looking for we will iterate through the list and select the ones that:
    
    1. contain: films/star- 
    2. are not in list we have previously created (to avoid duplicates)

In [51]:
of_links = []
for x in links:
    if 'films/star-' in x and x not in of_links:
        of_links.append(x)
of_links

['https://www.starwars.com/films/star-wars-episode-i-the-phantom-menace',
 'https://www.starwars.com/films/star-wars-episode-ii-attack-of-the-clones',
 'https://www.starwars.com/films/star-wars-episode-iii-revenge-of-the-sith',
 'https://www.starwars.com/films/star-wars-episode-iv-a-new-hope',
 'https://www.starwars.com/films/star-wars-episode-v-the-empire-strikes-back',
 'https://www.starwars.com/films/star-wars-episode-vi-return-of-the-jedi',
 'https://www.starwars.com/films/star-wars-episode-vii-the-force-awakens',
 'https://www.starwars.com/films/star-wars-episode-viii-the-last-jedi',
 'https://www.starwars.com/films/star-wars-episode-ix-the-rise-of-skywalker']

We can see that so far we have 6 movies and 3 Disney versions; as we said before we will just work with the movies so we need to take the elements 0-5 from the list.

In [52]:
official_links = of_links[:6]
official_links

['https://www.starwars.com/films/star-wars-episode-i-the-phantom-menace',
 'https://www.starwars.com/films/star-wars-episode-ii-attack-of-the-clones',
 'https://www.starwars.com/films/star-wars-episode-iii-revenge-of-the-sith',
 'https://www.starwars.com/films/star-wars-episode-iv-a-new-hope',
 'https://www.starwars.com/films/star-wars-episode-v-the-empire-strikes-back',
 'https://www.starwars.com/films/star-wars-episode-vi-return-of-the-jedi']

## Including the information on the `star_wars_final` data frame

### `Official Site` : making links clickables

We will create a new column `Official Site` to include our links, then we will apply the *make_clickable* function, which we can find in section 1.2 Define Functions.

In [64]:
star_wars_final['Official Site'] = official_links
starwars_links = star_wars_final.style.format({'Official Site': make_clickable})

In [66]:
display(starwars_links)

,Episode,Title,Characters,Planets,Starships,Vehicles,Official Site
0,1,The Phantom Menace,"C-3PO, R2-D2, Obi-Wan Kenobi, Jabba Desilijic Tiure, Anakin Skywalker, Yoda, Palpatine, Nute Gunray, Jar Jar Binks, Watto, Shmi Skywalker, Ayla Secura, Mace Windu, Ki-Adi-Mundi, Kit Fisto, Plo Koon, Mas Amedda, Padmé Amidala, Qui-Gon Jinn, Finis Valorum, Roos Tarpals, Rugor Nass, Ric Olié, Sebulba, Quarsh Panaka, Darth Maul, Dud Bolt, Gasgano, Ben Quadinaros, Eeth Koth, Adi Gallia, Saesee Tiin, Yarael Poof, Ratts Tyerell","Tatooine, Naboo, Coruscant","Naboo fighter, Droid control ship, Naboo Royal Starship, Scimitar, Republic Cruiser","Vulture Droid, Multi-Troop Transport, Armored Assault Tank, Single Trooper Aerial Platform, C-9979 landing craft, Tribubble bongo, Sith speeder",https://www.starwars.com/films/star-wars-episode-i-the-phantom-menace
1,2,Attack of the Clones,"C-3PO, R2-D2, Owen Lars, Beru Whitesun lars, Obi-Wan Kenobi, Anakin Skywalker, Yoda, Palpatine, Boba Fett, Nute Gunray, Jar Jar Binks, Watto, Shmi Skywalker, Ayla Secura, Mace Windu, Ki-Adi-Mundi, Kit Fisto, Plo Koon, Mas Amedda, Gregar Typho, Cordé, Cliegg Lars, Poggle the Lesser, Luminara Unduli, Barriss Offee, Dormé, Dooku, Bail Prestor Organa, Jango Fett, Zam Wesell, Dexter Jettster, Lama Su, Taun We, Jocasta Nu, R4-P17, Wat Tambor, San Hill, Shaak Ti, Sly Moore, Padmé Amidala","Tatooine, Naboo, Coruscant, Kamino, Geonosis","Slave 1, Naboo fighter, J-type diplomatic barge, AA-9 Coruscant freighter, Jedi starfighter, H-type Nubian yacht, Droid control ship, Republic Assault ship, Solar Sailer","Sand Crawler, Zephyr-G swoop bike, Koro-2 Exodrive airspeeder, XJ-6 airspeeder, LAAT/i, LAAT/c, AT-TE, SPHA, Flitknot speeder, Neimoidian shuttle, Geonosian starfighter",https://www.starwars.com/films/star-wars-episode-ii-attack-of-the-clones
2,3,Revenge of the Sith,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Leia Organa, Owen Lars, Beru Whitesun lars, Obi-Wan Kenobi, Wilhuff Tarkin, Chewbacca, Raymus Antilles, Anakin Skywalker, Yoda, Palpatine, Nute Gunray, Ayla Secura, Mace Windu, Ki-Adi-Mundi, Kit Fisto, Plo Koon, Poggle the Lesser, Luminara Unduli, Dooku, Bail Prestor Organa, R4-P17, Shaak Ti, Sly Moore, Padmé Amidala, Eeth Koth, Adi Gallia, Saesee Tiin, Grievous, Tarfful, Tion Medon","Alderaan, Tatooine, Naboo, Coruscant, Dagobah, Utapau, Mustafar, Kashyyyk, Polis Massa, Mygeeto, Felucia, Cato Neimoidia, Saleucami","CR90 corvette, Jedi starfighter, Droid control ship, Trade Federation cruiser, Theta-class T-2c shuttle, Republic attack cruiser, Naboo star skiff, Jedi Interceptor, arc-170, Belbullab-22 starfighter, V-wing, Banking clan frigate","LAAT/i, AT-TE, Neimoidian shuttle, Vulture Droid, Tsmeu-6 personal wheel bike, Emergency Firespeeder, Droid tri-fighter, Oevvaor jet catamaran, Raddaugh Gnasp fluttercraft, Clone turbo tank, Corporate Alliance tank droid, Droid gunship, AT-RT",https://www.starwars.com/films/star-wars-episode-iii-revenge-of-the-sith
3,4,A New Hope,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Leia Organa, Owen Lars, Beru Whitesun lars, R5-D4, Biggs Darklighter, Obi-Wan Kenobi, Wilhuff Tarkin, Chewbacca, Han Solo, Greedo, Jabba Desilijic Tiure, Wedge Antilles, Jek Tono Porkins, Raymus Antilles","Alderaan, Yavin IV, Tatooine","CR90 corvette, Star Destroyer, Sentinel-class landing craft, Death Star, Millennium Falcon, Y-wing, X-wing, TIE Advanced x1","Sand Crawler, T-16 skyhopper, X-34 landspeeder, TIE/LN starfighter",https://www.starwars.com/films/star-wars-episode-iv-a-new-hope
4,5,The Empire Strikes Back,"Luke Skywalker, C-3PO, R2-D2, Darth Vader, Leia Organa, Obi-Wan Kenobi, Chewbacca, Han Solo, Wedge Antilles, Yoda, Palpatine, Boba Fett, Lando Calrissian, IG-88, Bossk, Lobot","Dagobah, Hoth, Bespin, Ord Mantell","Star Destroyer, Millennium Falcon, Y-wing, X-wing, Slave 1, Executor, Imperial shuttle, EF76 Nebulon-B escort frigate, Rebel transport","TIE/LN starfighter, TIE bomber, AT-AT, AT-ST, Snowspeeder, Storm IV Twin-Pod cloud car",https://www.star

### `Thumbnails`: including pictures in a data frame.

To include the pictures in the data frame we will need to create a column `Thumbnails`. 

Things to take into account:
    1. We can not have indexed the Data Frame beforehand
    2. We have to convert the string of the link into an html tag (with the path_to_html function in section 1.2 Define Functions.)
    3. We have to set a column width for the display 
    4. It gives sth tons of problems to show the image so you might need to restart your kernel a lot.

In [54]:
#Option 1
#

# images = ['https://lumiere-a.akamaihd.net/v1/images/Star-Wars-Phantom-Menace-I-Poster_f5832812.jpeg?region=0%2C250%2C678%2C340&width=320',
#  'https://lumiere-a.akamaihd.net/v1/images/Star-Wars-Attack-Clones-II-Poster_53baa2e7.jpeg?region=0%2C188%2C678%2C340&width=320',
#  'https://lumiere-a.akamaihd.net/v1/images/Star-Wars-Revenge-Sith-III-Poster_646108ce.jpeg?region=0%2C356%2C746%2C374&width=320',
#  'https://lumiere-a.akamaihd.net/v1/images/Star-Wars-New-Hope-IV-Poster_c217085b.jpeg?region=40%2C219%2C586%2C293&width=320',
#  'https://lumiere-a.akamaihd.net/v1/images/Star-Wars-Empire-Strikes-Back-V-Poster_878f7fce.jpeg?region=25%2C299%2C612%2C306&width=320',
#  'https://lumiere-a.akamaihd.net/v1/images/Star-Wars-Return-Jedi-VI-Poster_a10501d2.jpeg?region=9%2C210%2C624%2C312&width=320']

# star_wars_final['Promotional Poster'] = images

# pd.set_option('display.max_colwidth', -1)
# star_wars_html = HTML(star_wars_final.to_html(escape=False ,formatters=dict(image=path_to_html)))


In [55]:
# #Option 2
# star_wars_final['Promotional Poster'] = promo_poster_ordered 
# pd.set_option('display.max_colwidth', -1)
# star_wars_final = HTML(star_wars_final.to_html(escape=False ,formatters=dict(image=path_to_html)))

# <img src="https://fontmeme.com/permalink/200220/766f4b73d6d29fe8b29514066e514636.png" alt="star-wars-font" border="0"></a>

In [56]:
# star_wars_html